<a href="https://colab.research.google.com/github/acpuche/web_scraping_genai/blob/parsing/web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setup

In [6]:
#Requirements
#!pip install asyncio==3.4.3
#!pip install Crawl4AI==0.6.3
#!pip install Unidecode==1.4.0

#!pip install fastmcp==2.9.0

# run playwright install

!pip install -r /content/requirements.txt -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.0/93.0 kB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the s

In [10]:
!playwright install

171.6 MiB [] 0% 11.4s171.6 MiB [] 0% 50.1s171.6 MiB [] 0% 25.5s171.6 MiB [] 0% 16.1s171.6 MiB [] 0% 9.1s171.6 MiB [] 1% 5.9s171.6 MiB [] 1% 5.3s171.6 MiB [] 2% 4.3s171.6 MiB [] 3% 3.7s171.6 MiB [] 4% 3.2s171.6 MiB [] 5% 3.0s171.6 MiB [] 5% 3.2s171.6 MiB [] 6% 3.1s171.6 MiB [] 6% 3.2s171.6 MiB [] 6% 3.3s171.6 MiB [] 7% 3.2s171.6 MiB [] 8% 3.3s171.6 MiB [] 8% 3.2s171.6 MiB [] 9% 3.2s171.6 MiB [] 9% 3.3s171.6 MiB [] 10% 3.2s171.6 MiB [] 10% 3.1s171.6 MiB [] 11% 3.2s171.6 MiB [] 11% 3.1s171.6 MiB [] 12% 3.1s171.6 MiB [] 13% 3.0s171.6 MiB [] 14% 3.0s171.6 MiB [] 14% 2.9s171.6 MiB [] 15% 2.8s171.6 MiB [] 16% 2.8s171.6 MiB [] 17% 2.7s171.6 MiB [] 18% 2.6s171.6 MiB [] 19% 2.6s171.6 MiB [] 19% 2.5s171.6 MiB [] 19% 2.6s171.6 MiB [] 20% 2.5s171.6 MiB [] 21% 2.5s171.6 MiB [] 22% 2.4s171.6 MiB [] 22% 2.5s171.6 MiB [] 22% 2.6s171.6 MiB [] 23% 2.4s171.6 MiB [] 24% 2.4s171.6 MiB [] 25% 2.3s171.6 MiB [] 26% 2.3s171.6 MiB [] 27% 2.2s171.6 MiB [] 28% 2.2s171.6 MiB [] 29% 2.1s171.6 MiB [] 30% 2.0s171.6 Mi

## Paso 1
Muestreo de urls internas de un sitio de e-commerce, como salida se genera un json con el nombre del comercio y la hora de generación.

La busqueda está limitada por el parámetro maxlen=50 para reducir costos al momento de usar los modelos de lenguaje.




In [ ]:
"https://www.amazon.com/"
"https://www.mercadolibre.com.co/"
"https://www.shein.com.co/"
"https://www.farmatodo.com.co/"

In [8]:
e_commerce_url="https://www.amazon.com/"

In [11]:
import json
import asyncio
import datetime
from unidecode import unidecode
from collections import deque
from typing import Dict, List, Set
from crawl4ai.models import CrawlResult
from crawl4ai import AsyncWebCrawler
from urllib.parse import urljoin, urlparse, urlunparse

class SimpleWebsiteScraper:
    def __init__(self, crawler: AsyncWebCrawler):
        self.crawler = crawler
        self.base_url = None

    def is_valid_internal_link(self, link: str) -> bool:
        if not link or link.startswith('#'):
            return False

        parsed_base = urlparse(self.base_url)
        parsed_link = urlparse(link)

        return (parsed_base.netloc == parsed_link.netloc and
                parsed_link.path not in ['', '/'] and
                parsed_link.path.startswith(parsed_base.path))

    def normalize_url(self, url: str) -> str:
        parsed = urlparse(url)
        # Remove any fragments
        parsed = parsed._replace(fragment='')
        # Ensure the path doesn't end with a slash unless it's the root
        if parsed.path.endswith('/') and len(parsed.path) > 1:
            parsed = parsed._replace(path=parsed.path.rstrip('/'))
        return urlunparse(parsed)

    def join_url(self, base: str, url: str) -> str:
        joined = urljoin(base, url)
        parsed_base = urlparse(self.base_url)
        parsed_joined = urlparse(joined)

        # Ensure the joined URL starts with the base path
        if not parsed_joined.path.startswith(parsed_base.path):
            # If it doesn't, prepend the base path
            new_path = parsed_base.path.rstrip('/') + '/' + parsed_joined.path.lstrip('/')
            parsed_joined = parsed_joined._replace(path=new_path)

        return urlunparse(parsed_joined)

    async def scrape(self, start_url: str, max_depth: int) -> Dict[str, CrawlResult]:
        self.base_url = self.normalize_url(start_url)
        results: Dict[str, CrawlResult] = {}
        queue: deque = deque([(self.base_url, 0)],maxlen=50)
        visited: Set[str] = set()

        while queue:

          current_url, current_depth = queue.popleft()
          result = await self.crawler.arun(current_url)

          if current_url in visited or current_depth > max_depth:
            continue

          visited.add(current_url)
          result = await self.crawler.arun(current_url)

          internal_links_test = result.links.get('internal', [])
          for link_test in internal_links_test:
            link_test['text']=unidecode(link_test['text'])

          if result.success:
            results[current_url] = result

            if current_depth < max_depth:
              internal_links = result.links.get('internal', [])
              for link in internal_links:
                full_url = self.join_url(current_url, link['href'])
                normalized_url = self.normalize_url(full_url)
                if self.is_valid_internal_link(normalized_url) and normalized_url not in visited:
                  queue.append((normalized_url, current_depth + 1))

        return results

async def main(start_url: str, depth: int):
    all_results=[]

    async with AsyncWebCrawler() as crawler:
        scraper = SimpleWebsiteScraper(crawler)
        results = await scraper.scrape(start_url, depth)

    # Get the current date and time
    now = datetime.datetime.now()

    # Format the date and time as a string (e.g., "2025-06-23_14-30-00")
    formatted_datetime = now.strftime("%Y-%m-%d_%H-%M-%S")

    # Para varias páginas, hay inconveniente con el append del json y posterior uso.
    print(f"Crawled {len(results)} pages:")
    for url, result in results.items():
        print(f"- {url}: {len(result.links.get('internal', []))} internal links, {len(result.links.get('external', []))} external links")
        print(f"Session: {result.session_id}", )

        """
        with open(f"{start_url.split('.')[1]}_{formatted_datetime}.json", "w") as f:
          all_results.append(result.links)
          json.dump(all_results, f, indent=4)
        """
    json_name = f"{start_url.split('.')[1]}_{formatted_datetime}.json"

    with open(json_name, "w") as f:
      json.dump(result.links, f, indent=4)

    return results, json_name

if __name__ == "__main__":
    start_url = e_commerce_url
    depth = 1
    results, json_name=await(main(start_url, depth))

[INIT].... → Crawl4AI 0.6.3 

[FETCH]... ↓ https://www.amazon.com/                                                                              |
✓ | ⏱: 1.55s 

[SCRAPE].. ◆ https://www.amazon.com/                                                                              |
✓ | ⏱: 0.04s 

[COMPLETE] ● https://www.amazon.com/                                                                              |
✓ | ⏱: 1.62s 

[FETCH]... ↓ https://www.amazon.com/                                                                              |
✓ | ⏱: 0.95s 

...

[COMPLETE] ● https://www.amazon.com/gp/help/customer/display.html/ref=footer_privacy?ie=UTF8&nodeId=468496        |
✓ | ⏱: 2.63s 

[FETCH]... ↓ https://www.amazon.com/gp/help/customer/display.html/ref=footer_privacy?ie=UTF8&nodeId=468496        |
✓ | ⏱: 1.48s 

[SCRAPE].. ◆ https://www.amazon.com/gp/help/customer/display.html/ref=footer_privacy?ie=UTF8&nodeId=468496        |
✓ | ⏱: 0.28s 

[COMPLETE] ● https://www.amazon.com/gp/help/customer/display.html/ref=footer_privacy?ie=UTF8&nodeId=468496        |
✓ | ⏱: 1.77s 

Crawled 3 pages:
- https://www.amazon.com/: 2 internal links, 0 external links
Session: None
- https://www.amazon.com/gp/help/customer/display.html/ref=footer_cou?ie=UTF8&nodeId=508088: 142 internal links, 22 external links
Session: None
- https://www.amazon.com/gp/help/customer/display.html/ref=footer_privacy?ie=UTF8&nodeId=468496: 184 internal links, 22 external links
Session: None


## Paso 2
#### Uso de LLM para filtrado de contenido, se parte de una lista de urls del sitio de e-commerce para filtrar el contenido de interés: productos.

In [12]:
!pip install -q -U google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 5.8 MB/s eta 0:00:00


#### Costo por millón de tokens de entrada y salida para el modelo Gemini 2.0 flash

In [47]:
gemini_2_0_flash_input_cost = 0.10
gemini_2_0_flash_output_cost = 0.40

#Utils

#### Parseo de la respuesta del modelo para exporotar en formato json


In [27]:
def parse_to_dictionary(text_to_parse):
  """
  método que parsea un string a un diccionario cono diferentes estrategias
  """
  parsing_strategies = [
      lambda x: eval(x), # Generic eval for python dictionaries
      lambda x: json.loads(re.sub(r'\\n', '\n', re.sub(r'\\"', '"', re.sub(r'\\\\', '\\',re.search(r'```json\\n(.*?)```', s, re.DOTALL).group(1))))),
      lambda x: json.loads(json.loads(x.strip('"')).replace("```json\\n", "").replace("\\n```", "")),
      lambda x: json.loads(x.replace("\n", "").strip(".\n")), # Clean and parse JSON directly
      lambda x: json.loads(re.search(r'```json\n(.*?)\n```', x, re.DOTALL).group(1)),
      lambda x: eval(re.search(r"```json\n(.*?)\n```", x, re.DOTALL).group(1)),
      lambda x: json.loads(re.search(r"\{.*\}", x, re.DOTALL.group(0))), # Extract shortest JSON-like object with eval
      lambda x: eval(re.search(r"\{.*\}", x, re.DOTALL).group(0)), #Extract JSON-like content usinng eval.
      lambda x: json.loads(re.search(r"```json\s*(\{.*?\})\s*```", x, re.DOTALL).group(1)), # Parse JSON in a ```json``` block
  ]

  for strategy in parsing_strategies:
    try:
      return strategy(text_to_parse)
    except Exception:
            continue

"""
if isinstance(parsed_json, str): # si ninguna de las estrategias de parseo funciona
  text = re.sub(r"```json", "", parsed_json)
  # Elimina todas las secuencias de \n
  text = re.sub(r"\\n", "", text)
  # Elimina todas las secuencias de \n
  list_from_text = re.sub(r"\n", "", text)
  # (Opcional) elimina múltiples espacios si los hay
  list_from_text = [re.sub(r"\s{2,}", " ", text)]
"""


'\nif isinstance(parsed_json, str): # si ninguna de las estrategias de parseo funciona\n  text = re.sub(r"```json", "", parsed_json)\n  # Elimina todas las secuencias de \n\n  text = re.sub(r"\\n", "", text)\n  # Elimina todas las secuencias de \n\n  list_from_text = re.sub(r"\n", "", text)\n  # (Opcional) elimina múltiples espacios si los hay\n  list_from_text = [re.sub(r"\\s{2,}", " ", text)]\n'

Cáulculo del costo por petición a la API

In [51]:
def computing_cost(response, input_cost, output_cost):
  """
  Input and output cost per million tokens
  """
  num_input_tokens = response.usage_metadata.prompt_token_count
  num_output_tokens = response.usage_metadata.candidates_token_count
  total_cost_per_call=((num_input_tokens)/1000000)*input_cost+((num_output_tokens)/1000000)*output_cost
  print(f'El costo total de la petición fue de: {round(total_cost_per_call,4)} usd \nTokens de entrada:{num_input_tokens}\nTokens de salida:{num_output_tokens}')

#### Diseño y ejecución del modelo de lenguaje para filtrado de contenido textual, conservando la estructura inicial de la información.

In [14]:
import json
from google import genai
from google.genai import types
from google.colab import userdata

input_file = json_name # Del paso 1

with open(input_file) as json_file:
  json_contents = json_file.read()

client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY_1"))

model_id = "gemini-2.0-flash"

system_prompt = """
Eres un desarrollador experimentado que recibe un archivo JSON con información de un e-commerce con los siguientes atributos:
href, text, title y base_domain. Solo debes tomar la información contenida en "internal".

Tu tarea es generar un JSON limpio removiendo los bloques de información que no hagan referencia a productos.
Remueve los signos de puntuación por la letra sin tilde.

Incluye contenido relacionado con productos, usualmente se encuentran por categorías. Por ejemplo:
- Salud
- Belleza
- Vehículos
- Alimentos
- Moda

Excluye contenido que tenga información relacionada con:
- Políticas
- Terminos y condiciones
- Redes sociales

"""
prompt = f"""
Ten en cuenta que no puedes agregar otras secciones diferentes a las mencionadas en la salida.

Entrada:
"texto: {json_contents}"

la salida debe ser un JSON que mantenga la estructura del archivo de entrada.
No debes agregar información adicional.

Salida:
"href": "url",
"text": "url description",
"title": "",
"base_domain": "site domain"
"""

response = client.models.generate_content(
    model=model_id,
    contents=prompt,
    config=types.GenerateContentConfig(
        system_instruction=system_prompt,
        temperature=0
    )
)

computing_cost(response,gemini_2_0_flash_input_cost,gemini_2_0_flash_output_cost)

Manipulación de la respuesta

In [28]:
print(response.text)

parsed_response = parse_to_dictionary(response.text)

filtered_json_name = f"{input_file.split('.')[0]}_cleaned.json"

with open(filtered_json_name, "w") as f:
  json.dump(parsed_response, f, indent=4, ensure_ascii=False)
  print(f"{filtered_json_name} ha sido guardado")

#print(f"Se ha filtrado el contenido con éxito. El número de sitios posibles a scrapear de {input_file.split('.')[0]} :{len(parsed_response['internal'])}")

```json
{
  "internal": [
    {
      "href": "https://www.amazon.com/haul/store?ref_=nav_cs_hul_disb",
      "text": "Amazon Haul",
      "title": "",
      "base_domain": "amazon.com"
    },
    {
      "href": "https://health.amazon.com/prime?ref_=nav_cs_all_health_ingress_onem_h",
      "text": "Medical Care",
      "title": "",
      "base_domain": "amazon.com"
    },
    {
      "href": "https://www.amazon.com/luxurystores/saks?ref_=nav_cs_saks_disc",
      "text": "Saks",
      "title": "",
      "base_domain": "amazon.com"
    },
    {
      "href": "https://www.amazon.com/gp/bestsellers/?ref_=nav_cs_bestsellers",
      "text": "Best Sellers",
      "title": "",
      "base_domain": "amazon.com"
    },
    {
      "href": "https://www.amazon.com/Amazon_Basics?channel=discovbar&field-lbr_brands_browse-bin=AmazonBasics&ref_=nav_cs_amazonbasics",
      "text": "Amazon Basics",
      "title": "",
      "base_domain": "amazon.com"
    },
    {
      "href": "https://www.amazon.com/g

## Paso 3
Se toman las urls de producto identificadas en el paso anterior y extraen markdowns con la información de interés.

Se guardan cada uno de los archivos y también se apilan en la lista *raw_output*

In [29]:
import re
import datetime
import asyncio
from crawl4ai import AsyncWebCrawler

input_file = filtered_json_name # del paso 2

with open(input_file) as json_file:
  json_contents = json_file.read()

parsed_json = parse_to_dictionary(json_contents)

async def main(url):
    # Create an instance of AsyncWebCrawler
    async with AsyncWebCrawler() as crawler:
        # Run the crawler on a URL
        result = await crawler.arun(url)

        # Get the current date and time
        now = datetime.datetime.now()

        # Format the date and time as a string (e.g., "2025-06-23_14-30-00")
        formatted_datetime = now.strftime("%Y-%m-%d_%H-%M-%S")

        # Save the markdown to a file
        if result.success:
            with open(url.split('.')[1] + '_' + formatted_datetime + '.md', "w", encoding="utf-8") as f:
                f.write(result.markdown.raw_markdown)
            print(f"Markdown saved to {url.split('.')[1]}_{formatted_datetime}")
        else:
            print(f"Failed to crawl: {result.error_message}")
    return result

# Run the async main function
raw_output = []

for item in range(len(parsed_json['internal'])):
  output = await(main(url=parsed_json['internal'][item-1]['href'])) #crawl4ai.models.CrawlResultContainer
  raw_output.append(output)

[INIT].... → Crawl4AI 0.6.3 

[FETCH]... ↓ https://www.amazon.com/gp/browse.html?node=12653393011&ref_=footer_usrenew                           |
✓ | ⏱: 7.12s 

[SCRAPE].. ◆ https://www.amazon.com/gp/browse.html?node=12653393011&ref_=footer_usrenew                           |
✓ | ⏱: 1.42s 

[COMPLETE] ● https://www.amazon.com/gp/browse.html?node=12653393011&ref_=footer_usrenew                           |
✓ | ⏱: 8.56s 

Markdown saved to amazon_2025-06-30_16-16-26


...

[INIT].... → Crawl4AI 0.6.3 

[FETCH]... ↓ https://www.amazon.com/gp/browse.html?node=14498690011&ref_=amzn_nav_ftr_swa                         |
✓ | ⏱: 1.91s 

[SCRAPE].. ◆ https://www.amazon.com/gp/browse.html?node=14498690011&ref_=amzn_nav_ftr_swa                         |
✓ | ⏱: 0.20s 

[COMPLETE] ● https://www.amazon.com/gp/browse.html?node=14498690011&ref_=amzn_nav_ftr_swa                         |
✓ | ⏱: 2.12s 

Markdown saved to amazon_2025-06-30_16-20-37


## Paso 4
Se usa un modelo de lenguaje para tomar la información extraida de los markdown de las páginas de producto y extraer la información de interés:



```
"ID": "identificación del producto",
"title": "nombre del producto",
"Precio": "precio del producto",
"imagen_url": "url de la imagen del producto"
"descripción": "descripción del producto"
"base_domain": "site domain"
"base_product_uls": "url del producto"
```



In [49]:
import json
from google import genai
from google.genai import types
from google.colab import userdata

client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY_1"))

model_id = "gemini-2.0-flash"

system_prompt = """
Eres un desarrollador experimentado que recibe un archivo markdown con información de un e-commerce con información de productos.
dentro de las características extraidas se encuentra la sección a la que pertenece y nombre, url de la imagen, url del producto, descripción y precio.

Tu tarea es generar un JSON limpio removiendo los bloques de información que no hagan referencia a productos.
Remueve los signos de puntuación por la letra sin tilde.

Incluye contenido relacionado con productos, usualmente se encuentran por categorías. Por ejemplo:
- Salud
- Belleza
- Vehículos
- Alimentos
- Moda

Excluye contenido que tenga información relacionada con:
- Políticas
- Terminos y condiciones
- Redes sociales

"""
prompt = f"""
Ten en cuenta que no puedes agregar otras secciones diferentes a las mencionadas en la salida.

Entrada:
"texto: {raw_output[1]}"

la salida debe ser un JSON que contenga la información de cada producto encontrado.
No debes agregar información adicional.

Salida:
"ID": "identificación del producto",
"title": "nombre del producto",
"Precio": "precio del producto",
"imagen_url": "url de la imagen del producto"
"descripción": "descripción del producto"
"base_domain": "site domain"
"base_product_uls": "url del producto"
"""

response = client.models.generate_content(
    model=model_id,
    contents=prompt,
    config=types.GenerateContentConfig(
        system_instruction=system_prompt,
        temperature=0
    )
)

In [52]:
computing_cost(response,gemini_2_0_flash_input_cost,gemini_2_0_flash_output_cost)

El costo total de la petición fue de: 0.0385 usd 
Tokens de entrada:364387
Tokens de salida:5085


In [53]:
print(response.text)

parsed_response = parse_to_dictionary(response.text)

products_file_json = f"{input_file.split('.')[0]}_products.json"

with open(products_file_json, "w") as f:
  #f.write(response.to_json_dict)
  json.dump(parsed_response, f, indent=4, ensure_ascii=False)
  print(f"{products_file_json} ha sido guardado")

```json
[
  {
    "ID": "B0DS6BLC74",
    "title": "Tribal Print Off Shoulder Shirred Maxi Dress Bell Sleeve Side Split Vacation Dress",
    "Precio": "11.49",
    "imagen_url": "https://m.media-amazon.com/images/I/61cy9l5YwXL._SL300_.jpg",
    "descripcion": "Tribal Print Off Shoulder Shirred Maxi Dress Bell Sleeve Side Split Vacation Dress",
    "seccion": "Moda",
    "base_domain": "amazon.com",
    "base_product_url": "https://www.amazon.com/Tribal-Shoulder-Shirred-Sleeve-Vacation/dp/B0DS6BLC74"
  },
  {
    "ID": "B0DZ2FHBZL",
    "title": "Cut Out Dress for Womens Drawstring Ruched Sleeveless Mini Dresses",
    "Precio": "7.99",
    "imagen_url": "https://m.media-amazon.com/images/I/71Yt8PxJ+-L._SL300_.jpg",
    "descripcion": "Cut Out Dress for Womens Drawstring Ruched Sleeveless Mini Dresses",
    "seccion": "Moda",
    "base_domain": "amazon.com",
    "base_product_url": "https://www.amazon.com/Womens-Drawstring-Ruched-Sleeveless-Dresses/dp/B0DZ2FHBZL"
  },
  {
    "ID": "B0DY